In [ ]:
''' Two Inputs : (i) a directory containing all images, (ii) scenic scenario script 
    Output     : a list of image_file names that belongs to the scenario'''

import os
import scenic
from scenic.simulators.gta.nusc_query_api import NuscQueryAPI

directory = "/Users/edwardkim/Desktop/nuScenes_data/samples/CAM_FRONT"
scenic_script_path = 'examples/gta/test_boston_seaport.sc'
# image_filename = [file for file in os.listdir(directory) if file.endswith('.jpg')]

In [ ]:
# extract image labels
query = NuscQueryAPI(version = 'v1.0-trainval', dataroot='/Users/edwardkim/Desktop/nuScenes_data')
map_name = 'boston-seaport'

In [ ]:
def deg_to_rad(degrees):
    return (degrees + 360) % 360 # <---- need to consider NuScenes reference angle to Scenic's reference angle

In [ ]:
NUM_CAR = 2 # excluding ego

# construct scenario
scenario = scenic.scenarioFromFile(scenic_script_path)

In [ ]:
from scenic.core.scenarios import Scene
from scenic.core.vectors import Vector
from scenic.core.distributions import Constant
import math

NUM_CAR = 1 # excluding ego
imgs_in_scenario = []

# for loop starts here over all images
scenario_copy = scenario
obj_list = scenario_copy.dependencies

img_filename = 'n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151603512404.jpg'
label = query.get_img_data(img_filename)

if query.img_filename_to_location[img_filename] == map_name:
    
    # skip if the num car condition not met
    if (len(label['Vehicles']) >= NUM_CAR):
        scenario_deps = [dep for dep in obj_list]
        
        # extract ego's NuScenes label values
        egoCar = label['EgoCar']
        ego_position = egoCar['position']
        ego_heading = egoCar['heading']
        
        # condition ego obj's position and heading to NuScene's label values
        x_ego = scenario.egoObject
        x_ego.position.conditionTo(Vector(ego_position[0], ego_position[1]))
        x_ego.heading.conditionTo(Constant(ego_heading - math.pi/2))
        scenario_copy.egoObject = x_ego
        scenario_deps[0] = x_ego
        
        # extract NuScenes' non-ego car's labels
        otherVehicles = label['Vehicles']

        for i in range(NUM_CAR):
            scenic_car = scenario_deps[i+1] # +1 added to skip ego car
            nuscenes_car = otherVehicles[i]
            nusc_car_position = nuscenes_car['position']
            nusc_car_heading  = nuscenes_car['heading']
            
            # condition on nuscenes' position and heading values
            scenic_car.position.conditionTo(Vector(nusc_car_position[0], nusc_car_position[1]))
            scenic_car.heading.conditionTo(Constant(nusc_car_heading - math.pi/2))
            
            # save the conditioned scenic car_obj
            scenario_deps[i+1] = scenic_car
            
        scenario_copy.dependencies = tuple(scenario_deps)
        scene = scenario_copy.generate(maxIterations = 1)
        if isinstance(scene, Scene):
            print("in scenario: ", img_filename)
            print("NuScenes label: ", label)
            cfg = scenic.simulators.gta.interface.GTA.Config(scene)
            print("scenic sample: ")
            print(cfg.vehicles)
            print(cfg.time)
            print(cfg.weather)
            print(cfg.location)
            print(cfg.view_heading)
            imgs_in_scenario.append(img_filename)
            

In [ ]:
m = query.get_whole_map('boston-seaport')
print(m)

In [ ]:
label = query.get_img_data('n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151603512404.jpg')
print(label)